In [ ]:
"""
SQL Practice
Part 2: Baseball Data

"""

In [25]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('lahman.sqlite')

query = "SELECT * FROM sqlite_master;"

df_schema = pd.read_sql_query(query, conn)

df_schema.tbl_name.unique()

array(['allstarfull', 'appearances', 'awardsmanagers', 'awardsplayers',
       'awardssharemanagers', 'awardsshareplayers', 'batting',
       'battingpost', 'collegeplaying', 'divisions', 'fielding',
       'fieldingof', 'fieldingofsplit', 'fieldingpost', 'halloffame',
       'homegames', 'leagues', 'managers', 'managershalf', 'parks',
       'people', 'pitching', 'pitchingpost', 'salaries', 'schools',
       'seriespost', 'teams', 'teamsfranchises', 'teamshalf'],
      dtype=object)

#### 1. What was the total spent on salaries by each team, each year?

In [26]:
query = """
SELECT *
FROM salaries
GROUP BY teamID
"""
df_salaries = pd.read_sql_query(query, conn)
df_salaries

,ID,yearID,teamID,team_ID,lgID,playerID,salary
0,9335,1997,ANA,2238,AL,alicelu01,650000.0
1,10294,1998,ARI,2267,NL,adamsjo02,210000.0
2,1,1985,ATL,1918,NL,barkele01,870000.0
3,23,1985,BAL,1919,AL,boddimi01,625000.0
4,45,1985,BOS,1920,AL,armasto01,915000.0
5,70,1985,CAL,1921,AL,beniqju01,365000.0
6,98,1985,CHA,1922,AL,agostju01,147500.0
7,119,1985,CHN,1923,NL,bosleth01,215000.0
8,141,1985,CIN,1924,NL,bellbu01,751297.0
9,163,1985,CLE,1925,AL,ayalabe01,303333.0


In [27]:
df_schema.tbl_name.unique()

array(['allstarfull', 'appearances', 'awardsmanagers', 'awardsplayers',
       'awardssharemanagers', 'awardsshareplayers', 'batting',
       'battingpost', 'collegeplaying', 'divisions', 'fielding',
       'fieldingof', 'fieldingofsplit', 'fieldingpost', 'halloffame',
       'homegames', 'leagues', 'managers', 'managershalf', 'parks',
       'people', 'pitching', 'pitchingpost', 'salaries', 'schools',
       'seriespost', 'teams', 'teamsfranchises', 'teamshalf'],
      dtype=object)

#### 2. What is the first and last year played for each player? Hint: Create a new table from 'Fielding.csv'.

In [28]:
query = """
SELECT DISTINCT playerID,
        MAX(yearID)
            OVER (PARTITION BY playerID),
        MIN(yearID)
            OVER (PARTITION BY playerID)
FROM fielding
GROUP BY playerID, yearID
"""
df = pd.read_sql_query(query, conn)
df

,playerID,MAX(yearID)\n OVER (PARTITION BY playerID),MIN(yearID)\n OVER (PARTITION BY playerID)
0,aardsda01,2015,2004
1,aaronha01,1976,1954
2,aaronto01,1971,1962
3,aasedo01,1990,1977
4,abadan01,2003,2001
...,...,...,...
19486,zupofr01,1961,1957
19487,zuvelpa01,1991,1982
19488,zuverge01,1959,1951
19489,zwilldu01,1916,1910


#### 3. Who has played the most all star games?

In [29]:
query = """
SELECT playerID, COUNT(ID)
FROM allstarfull
GROUP BY playerID
ORDER BY COUNT(ID) DESC
"""
df = pd.read_sql_query(query, conn)
df

,playerID,COUNT(ID)
0,aaronha01,24
1,mayswi01,24
2,musiast01,24
3,mantlmi01,19
4,ripkeca01,19
...,...,...
1862,zachrpa01,1
1863,zakfr01,1
1864,zarilal01,1
1865,zernigu01,1


#### 4. Which school has generated the most distinct players? Hint: Create new table from 'CollegePlaying.csv'.

In [30]:
query = """
SELECT name_full, COUNT(playerID)
FROM collegeplaying
JOIN schools
ON schools.schoolID = collegeplaying.schoolID
GROUP BY playerID, name_full
ORDER BY COUNT(playerID) DESC
"""
df = pd.read_sql_query(query, conn)
df

,name_full,COUNT(playerID)
0,Washington College,9
1,St. Bonaventure University,6
2,Tufts University,6
3,Kalamazoo College,6
4,University of Notre Dame,6
...,...,...
7540,Treasure Valley Community College,1
7541,Los Angeles Pierce College,1
7542,"University of California, Santa Barbara",1
7543,University of Southern California,1


#### 5. Which players have the longest career? Assume that the debut and finalGame columns comprise the start and end, respectively, of a player's career. Hint: Create a new table from 'Master.csv'. Also note that strings can be converted to dates using the DATE function and can then be subtracted from each other yielding their difference in days.

In [31]:
query = """
SELECT playerID, DATE(finalgame_date) - DATE(debut_date)
FROM people
GROUP BY playerID
ORDER BY (DATE(finalgame_date) - DATE(debut_date)) DESC
"""
df = pd.read_sql_query(query, conn)
df

,playerID,DATE(finalgame_date) - DATE(debut_date)
0,altroni01,35.0
1,orourji01,32.0
2,minosmi01,31.0
3,olearch01,30.0
4,lathaar01,29.0
...,...,...
19873,winklbo99,NaN
19874,wrighal99,NaN
19875,yawketo99,NaN
19876,youngni99,NaN


In [ ]:
df_schema.tbl_name.unique()

#### 6. What is the distribution of debut months? Hint: Look at the DATE and EXTRACT functions.

In [35]:
query = """
SELECT MIN(DATE(debut_date)), MAX(DATE(debut_date))
FROM people
"""
df = pd.read_sql_query(query, conn)
df

,MIN(DATE(debut_date)),MAX(DATE(debut_date))
0,1871-05-04,2019-09-27


In [ ]:
df_schema.tbl_name.unique()

#### 7. What is the effect of table join order on mean salary for the players listed in the main (master) table? Hint: Perform two different queries, one that joins on playerID in the salary table and other that joins on the same column in the master table. You will have to use left joins for each since right joins are not currently supported with SQLalchemy.

In [8]:
query = """
SELECT AVG(salary)
FROM salaries
LEFT JOIN people
ON salaries.playerID = people.playerID

"""
df = pd.read_sql_query(query, conn)
df

,AVG(salary)
0,2.085634e+06


In [11]:
query = """
SELECT AVG(salary)
FROM people
LEFT JOIN salaries
ON people.playerID = salaries.playerID

"""
df = pd.read_sql_query(query, conn)
df

,AVG(salary)
0,2.085634e+06


In [10]:
query = """
SELECT *
FROM people

"""
df = pd.read_sql_query(query, conn)
df

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,bats,throws,debut,finalGame,retroID,bbrefID,birth_date,debut_date,finalgame_date,death_date
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,1981-12-27,2004-04-06,2015-08-23,None
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,...,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01,1934-02-05,1954-04-13,1976-10-03,None
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,R,R,1962-04-10,1971-09-26,aarot101,aaronto01,1939-08-05,1962-04-10,1971-09-26,1984-08-16
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,R,R,1977-07-26,1990-10-03,aased001,aasedo01,1954-09-08,1977-07-26,1990-10-03,None
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,L,L,2001-09-10,2006-04-13,abada001,abadan01,1972-08-25,2001-09-10,2006-04-13,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19873,zupofr01,1939.0,8.0,29.0,USA,CA,San Francisco,2005.0,3.0,25.0,...,L,R,1957-07-01,1961-05-09,zupof101,zupofr01,1939-08-29,1957-07-01,1961-05-09,2005-03-25
19874,zuvelpa01,1958.0,10.0,31.0,USA,CA,San Mateo,NaN,NaN,NaN,...,R,R,1982-09-04,1991-05-02,zuvep001,zuvelpa01,1958-10-31,1982-09-04,1991-05-02,None
19875,zuverge01,1924.0,8.0,20.0,USA,MI,Holland,2014.0,9.0,8.0,...,R,R,1951-04-21,1959-06-15,zuveg101,zuverge01,1924-08-20,1951-04-21,1959-06-15,2014-09-08
19876,zwilldu01,1888.0,11.0,2.0,USA,MO,St. Louis,1978.0,3.0,27.0,...,L,L,1910-08-14,1916-07-12,zwild101,zwilldu01,1888-11-02,1910-08-14,1916-07-12,1978-03-27
